In [1]:
import sys, os
import boto3

import pandas as pd

In [2]:
in_file_list = '/home/wb411133/temp/World_Bank_Global_3_Complete.csv'
out_folder = os.path.join(os.path.dirname(in_file_list), "FATHOM")
s3_bucket = 'wbg-geography01'
s3_prefix = 'FATHOM/v2023/'
s3_out = os.path.join('s3://',s3_bucket, s3_prefix)

in_files = pd.read_csv(in_file_list)
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(s3_bucket)

FileNotFoundError: [Errno 2] No such file or directory: '/home/wb411133/temp/World_Bank_Global_3_Complete.csv'

In [ ]:
# Find all files already copied
all_folders = []
for obj in my_bucket.objects.filter(Prefix=s3_prefix):
    all_folders.append(obj.key.split("/")[-2])


In [ ]:
processed_folders = list(set(all_folders))
delivered_folders = in_files['Layer'].values
sel_folders = [x for x in delivered_folders if not x in processed_folders]
sel_files = in_files.loc[in_files['Layer'].isin(sel_folders)].copy()
sel_files

In [ ]:
with open(os.path.join(out_folder, "aaa_download_upload_2.sh"), 'w') as out_file:
    out_file.write('#!/bin/bash\n')
    for idx, row in sel_files.iterrows():
        fathom_path = row['AWS_Path']
        local_folder = os.path.join(out_folder, row['Layer'])
        gost_folder = os.path.join(s3_out, row['Layer'])
        if not os.path.exists(local_folder):
            os.makedirs(local_folder)
        
        cur_out_folder = os.path.join(s3_prefix, row['Layer'])
        obj_count = 0
        for obj in my_bucket.objects.filter(Prefix=cur_out_folder):
            obj_count += 1
        print(f"{row['Layer']}: {obj_count}")
        if obj_count == 0:
            download_command = f'aws s3 sync --profile fathom {fathom_path} {local_folder}'
            upload_command = f'aws s3 sync {local_folder} {gost_folder}'
            remove_command = f'rm -R {local_folder}'

            out_file.write(download_command)
            out_file.write('\n')
            out_file.write(upload_command)
            out_file.write('\n')
            out_file.write(remove_command)
            out_file.write('\n')

In [ ]:
all_vals = []
for idx, row in in_files.iterrows():
    all_vals.append(row['Layer'].split('-'))

In [ ]:
xx = pd.DataFrame(all_vals, columns=['GLOBAL', "Size",'Offset','return','type','defense','depth','year','projection','v1','v2'])
xx.head()


In [ ]:
xx['projection'].value_counts()

In [ ]:
xx.loc[xx['projection'] == 'SSP1_2.6']['year'].value_counts()

# Download and upload specific scenarios

In [5]:
sel_scenario = 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0'
bucket = "fathom-products-global"
prefix = f"fathom-global/v3/{sel_scenario}"

local_folder = os.path.join("/home/wb411133/temp/FATHOM", sel_scenario)
fathom_path = f's3://{bucket}/{prefix}'
gost_folder = f"s3://wbg-geography01/FATHOM/v2023/{sel_scenario}"

if not os.path.exists(local_folder):
    os.makedirs(local_folder)

download_command = f'aws s3 sync --profile fathom {fathom_path} {local_folder}'
upload_command = f'aws s3 sync {local_folder} {gost_folder}'
remove_command = f'rm -R {local_folder}'

print(download_command)
print(upload_command)
print(remove_command)


aws s3 sync --profile fathom s3://fathom-products-global/fathom-global/v3/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0 /home/wb411133/temp/FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0
aws s3 sync /home/wb411133/temp/FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0 s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0
rm -R /home/wb411133/temp/FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0
